In [1]:
from glob import glob
import pandas as pd
import numpy as np
import os
import json
from collections import defaultdict

In [2]:
df_movies = pd.read_csv('dataset/tmdb_5000_movies.csv')
# df_movies.loc[:,['id', 'keywords', 'genres', 'title']]
# df_credits = pd.read_csv('dataset/tmdb_5000_credits.csv')

df_movies = df_movies[['budget', 'genres', 'keywords', 'original_language'
                      , 'overview', 'popularity', 'production_companies'
                     , 'production_countries', 'release_date', 'revenue'
                     , 'runtime', 'spoken_languages', 'tagline', 'title'
                     , 'vote_average', 'vote_count']]
display(df_movies.sort_values(by='release_date', ascending=False).head())
# display(df_credits.head())

,budget,genres,keywords,original_language,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,tagline,title,vote_average,vote_count
4255,0,"[{""id"": 35, ""name"": ""Comedy""}, {""id"": 10751, ""...",[],en,"In 1979, an Indian family moves to America wit...",0.710870,[],[],2017-02-03,0,102.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",It’s better to stand out than to fit in.,Growing Up Smith,7.4,7
3408,0,"[{""id"": 18, ""name"": ""Drama""}, {""id"": 10749, ""n...","[{""id"": 1415, ""name"": ""small town""}, {""id"": 46...",en,Set in a small town near the North Pole where ...,3.139294,[],[],2016-10-02,0,96.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",NaN,Two Lovers and a Bear,6.8,27
3302,8000000,"[{""id"": 18, ""name"": ""Drama""}]","[{""id"": 1650, ""name"": ""cook""}, {""id"": 6054, ""n...",en,A unique friendship develops when a little gir...,7.828459,"[{""name"": ""Envision Media Arts"", ""id"": 19456},...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2016-09-16,0,104.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",He was the one person she could always count on.,Mr. Church,7.0,129
4720,8500000,"[{""id"": 18, ""name"": ""Drama""}]","[{""id"": 2831, ""name"": ""slavery""}]",en,"Nat Turner, a former slave in America, leads a...",9.452808,"[{""name"": ""Phantom Four"", ""id"": 423}, {""name"":...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2016-09-09,15861566,120.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",The Untold Story of Nat Turner,The Birth of a Nation,6.5,178
3249,0,"[{""id"": 12, ""name"": ""Adventure""}]","[{""id"": 3405, ""name"": ""blow job""}, {""id"": 1173...",en,When his hard-earned kicks get snatched by a l...,3.467923,"[{""name"": ""Bystorm Films"", ""id"": 2903}, {""name...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2016-09-09,0,80.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",They aren't just shoes,Kicks,7.5,18


In [3]:
def convert_list(cell):
    """convert the json format to a list of categories"""
    kw_list = []
    for kw in json.loads(cell):
        kw_list.append(kw['name'])
    return kw_list


def larger_n(col, n):
    """filter the column"""
    keywords = defaultdict(int)
    for row in df_movies[col]:
        row = json.loads(row)
        for entry in row:
            keywords[entry['name']] += 1
    kw_cnt = sorted(keywords.items(), key=lambda x: -x[1])
    return [kw[0] for kw in kw_cnt if kw[1] >= n]


larger_kw = larger_n('keywords', 20)
larger_genre = larger_n('genres', 1000)


df_movies['keywords'] = df_movies['keywords'].apply(
    convert_list).apply(lambda cell: [kw for kw in cell if kw in larger_kw])
# df_movies['keywords'] = df_movies['keywords'].apply(convert_list)
df_movies['genres'] = df_movies['genres'].apply(convert_list).apply(
    lambda cell: [kw for kw in cell if kw in larger_genre])
df_movies

,budget,genres,keywords,original_language,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,tagline,title,vote_average,vote_count
0,237000000,[Action],"[future, space travel, romance, space, alien, ...",en,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,[Action],"[love of one's life, ship, aftercreditsstinger]",en,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]","At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500
2,245000000,[Action],"[spy, based on novel, secret agent, sequel]",en,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",A Plan No One Escapes,Spectre,6.3,4466
3,250000000,"[Action, Drama, Thriller]","[dc comics, terrorist, secret identity, superh...",en,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-07-16,1084939099,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",The Legend Ends,The Dark Knight Rises,7.6,9106
4,260000000,[Action],"[based on novel, space travel, princess, alien...",en,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-03-07,284139100,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]","Lost in our world, found in another.",John Carter,6.1,2124
5,258000000,[Action],"[amnesia, love of one's life, marvel comic, se...",en,The seemingly invincible Spider-Man goes up ag...,115.699814,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-01,890871626,139.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",The battle within.,Spider-Man 3,5.9,3576
6,260000000,[],"[hostage, magic, horse, musical, princess, ani...",en,When the kingdom's most wanted-and most charmi...,48.681969,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2010-11-24,591794936,100.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",They're taking adventure to new lengths.,Tangled,7.4,3330
7,280000000,[Action],"[marvel comic, sequel, superhero, based on com...",en,When Tony Stark tries to jumpstart a dormant p...,134.279229,"[{""name"": ""Marvel Studios"", ""id"": 420}, {""name...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2015-04-22,1405403694,141.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",A New Age Has Come.,Avengers: Age of Ultron,7.3,6767
8,250000000,[],"[witch, magic, werewolf]",en,"As Harry begins his sixth year at Hogwarts, he...",98.885637,"[{""name"": ""Warner Bros."", ""id"": 6194}, {""name""...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2009-07-07,933959197,153.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Dark Secrets Revealed,Harry Potter and the Half-Blood Prince,7.4,5293
9,250000000,[Action],"[dc comics, superhero, based on comic book, re...",en,Fearing the actions of a god-like Super Hero l...,155.790452,"[{""name"": ""DC Comics"", ""id"": 429}, {""name"": ""A...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2016-03-23,873260194,151.0,"[{""iso_639_

In [4]:
df_movies = df_movies.dropna(subset=['overview'])

In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer
v = TfidfVectorizer()
x = v.fit_transform(df_movies['overview'])

train_features = pd.DataFrame(x.toarray(), columns=v.get_feature_names())


## get dummies

In [6]:
df_movies = df_movies.reset_index(drop=True)

In [7]:
from sklearn.preprocessing import MultiLabelBinarizer

mlb = MultiLabelBinarizer()
genres_dummies = pd.DataFrame(mlb.fit_transform(
    df_movies['genres']), columns=mlb.classes_).add_prefix('genre_')
keywords_dummies = pd.DataFrame(mlb.fit_transform(
    df_movies['keywords']), columns=mlb.classes_).add_prefix('kw_')
df_new = pd.concat([df_movies.loc[:, ['budget', 'popularity']],
                    genres_dummies, keywords_dummies,
                    train_features], axis=1)

df_y = df_new.loc[:, df_new.columns.str.startswith('genre_')]
df_X = df_new.drop(
    df_new.columns[df_new.columns.str.startswith('genre_')], inplace=False, axis=1)

In [8]:
from sklearn.model_selection import train_test_split

train_X, test_X, train_y, test_y = train_test_split(df_X, df_y, test_size=0.33, random_state=42)

In [9]:
from skmultilearn.problem_transform import BinaryRelevance
from sklearn.ensemble import RandomForestClassifier

classifier = BinaryRelevance(RandomForestClassifier())
classifier.fit(train_X, train_y)
predictions = classifier.predict(test_X)

from sklearn.metrics import accuracy_score
accuracy_score(test_y, predictions)

0.2481060606060606